In [1]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [3]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from transformers import Trainer, TrainingArguments
from transformers import VisionEncoderDecoderModel, AutoProcessor
from datasets import load_dataset
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm_notebook as tqdm
from transformers import AdamW
import torch.nn.functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'
from jiwer import wer

In [4]:
# Load the model and processor
model = VisionEncoderDecoderModel.from_pretrained("qantev/trocr-large-spanish")
processor = AutoProcessor.from_pretrained("qantev/trocr-large-spanish")
tokenizer = TrOCRProcessor.from_pretrained('qantev/trocr-large-spanish')

config.json:   0%|          | 0.00/4.97k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


generation_config.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [5]:
model.load_state_dict(torch.load("/kaggle/input/large-trocr-8-epochs-post-process/model_weights_large_grayscale_contrast (1).pth"))

/tmp/ipykernel_30/3069459000.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/large-trocr-8-epochs-post-process/model_weig

<All keys matched successfully>

In [6]:
test_df = pd.read_csv('/kaggle/input/ai-of-god-3/Public_data/test.csv')

In [7]:
test_path = '/kaggle/input/ai-of-god-3/Public_data/test_images'

# Function to convert unique ID into correct image path
def construct_image_path(unique_id):
    parts = unique_id.split('_')
    page_number = parts[1]
    l_number = parts[3] 
    
    image_path = os.path.join(test_path, f'Page_{page_number}', f'L_{l_number}.png')
    return image_path

test_df['image_path'] = test_df['unique Id'].apply(construct_image_path)

# Check the resulting dataframe
print(test_df[[ 'image_path']].head())

                                          image_path
0  /kaggle/input/ai-of-god-3/Public_data/test_ima...
1  /kaggle/input/ai-of-god-3/Public_data/test_ima...
2  /kaggle/input/ai-of-god-3/Public_data/test_ima...
3  /kaggle/input/ai-of-god-3/Public_data/test_ima...
4  /kaggle/input/ai-of-god-3/Public_data/test_ima...


In [8]:
from torchvision import transforms

In [9]:
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-23): 24 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=False)
              (key): Linear(in_features=1024, out_features=1024, bias=False)
              (value): Linear(in_features=1024, out_features=1024, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Line

In [13]:
predictions = []

# Set the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to calculate gradients during inference
    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
        image_path = row['image_path']
        
        # Open the image as grayscale
        image = Image.open(image_path).convert("L")  # 'L' mode for grayscale
        
        # Transform the image to tensor format
        transform = transforms.ToTensor()
        image_tensor = transform(image)  # Shape: [1, H, W] for grayscale
        
        # Stack the grayscale image to 3 channels to match model input
        image_stacked = torch.cat([image_tensor] * 3, dim=0)  # Shape: [3, H, W]

        # Apply threshold transformation
        mask = image_stacked < 0.6875
        image_stacked[mask] = 0.0
        image_stacked[~mask] = 1.0

        # Reshape the image for batch processing
        image_stacked = image_stacked.unsqueeze(0)  # Add batch dimension, Shape: [1, 3, H, W]

        # Process the image using the processor without rescaling (as per your requirement)
        pixel_values = processor(image_stacked, return_tensors="pt", do_rescale=False).pixel_values.to(device)
        
        # Generate the output using the model
        outputs = model.generate(pixel_values)

        # Decode the generated output to get the predicted string
        predicted_string = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Append the results to predictions list
        predictions.append({
            'unique Id': row['unique Id'],  # Ensure 'unique Id' matches your column name
            'prediction': predicted_string
        })

# Create a DataFrame for storing predictions
predictions_df = pd.DataFrame(predictions)

Inference:   0%|          | 0/168 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1338: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [14]:
import re

def preprocess_transcription(text):
    """
    Preprocess the transcriptions based on Old Spanish characteristics.
    """
    # Handle interchangeable letters
    text = re.sub(r'(?<!\S)[uU](?!\S)', 'v', text)  # u -> v
    text = re.sub(r'(?<!\S)[vV](?!\S)', 'u', text)  # v -> u
    text = re.sub(r'(?<!\S)[fF](?!\S)', 's', text)  # f -> s
    text = re.sub(r'(?<!\S)[sS](?!\S)', 'f', text)  # s -> f
    
    # Remove accents
    text = re.sub(r'[áàäâ]', 'a', text)
    text = re.sub(r'[éèëê]', 'e', text)
    text = re.sub(r'[íìïî]', 'i', text)
    text = re.sub(r'[óòöô]', 'o', text)
    text = re.sub(r'[úùüû]', 'u', text)

    return text

In [15]:
predictions_df['postprocessed_transcription'] = predictions_df['prediction'].apply(preprocess_transcription)

In [16]:
def add_quotes(value):
    # Check if the value is not already enclosed in double quotes
    if not (isinstance(value, str) and value.startswith('"') and value.endswith('"')):
        return f'"{value}"'  # Add double quotes
    return value 

In [17]:
def remove_quotes(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Remove leading and trailing double quotes
        return value.strip('"')
    return value

In [18]:
predictions_df['prediction']=predictions_df['prediction'].apply(remove_quotes)
print(predictions_df)

    unique Id                              prediction  \
0     P_1_L_1  crio señor por sus rectifsimos y o cul   
1     P_1_L_2       tos juyrios de darme tiempo, para   
2     P_1_L_3        que en ruestra educacion lo mani   
3     P_1_L_4    tellalle, pues me hallo tantos meses   
4     P_1_L_5       ha tendida en esta cama, á vna en   
..        ...                                     ...   
163  P_7_L_20         Ciceron del Noble, y creed, que   
164  P_7_L_21     son talentos de que le aueys de dar   
165  P_7_L_22        cuenta muy estrecha, y carga con   
166  P_7_L_23       que os ha obligado á ser defenfor   
167  P_7_L_24     de su Fū y Religion, accerrimo per-   

                postprocessed_transcription  
0    crio señor por sus rectifsimos y o cul  
1         tos juyrios de darme tiempo, para  
2          que en ruestra educacion lo mani  
3      tellalle, pues me hallo tantos meses  
4         ha tendida en esta cama, a vna en  
..                                     

In [19]:
predictions_df = predictions_df.drop('prediction', axis=1)

In [20]:
predictions_df = predictions_df.rename(columns={'postprocessed_transcription': 'prediction'})

In [21]:
predictions_df.head()

,unique Id,prediction
0,P_1_L_1,crio señor por sus rectifsimos y o cul
1,P_1_L_2,"tos juyrios de darme tiempo, para"
2,P_1_L_3,que en ruestra educacion lo mani
3,P_1_L_4,"tellalle, pues me hallo tantos meses"
4,P_1_L_5,"ha tendida en esta cama, a vna en"


In [22]:
predictions_df.to_csv('submission.csv', index=False)